In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime
from openai import OpenAI
import pymssql
import re
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from threading import Thread
import functools
from tableone import TableOne
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu as mwu
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.metrics import roc_curve, auc
import random
from threading import Thread
import functools
import anthropic

conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
anthropic_api_key="" ##enter anthropic api key

In [ ]:
rand_df=pd.read_csv('20241023_eGFR_synthetic_20000.csv')

In [ ]:
usercontent_global1='''Calculate estimated glomerular filtration rate (eGFR) for the person below.

'''
usercontent_global2='''

Please answer exactly in the format below, without blank lines, and no further information or answer is required.
eGFR=(in mL/min/1.73 m², round to two decimal places)'''

input_text=rand_df['input_text']
eid=rand_df['eid']

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

@timeout(0.1)
def ChatGPT_main(i,temper,responsenum):
    if input_text[i]!='':
        try:
            usercontent=usercontent_global1
            usercontent+=input_text[i]
            usercontent+=usercontent_global2

            client = anthropic.Anthropic(
                # defaults to os.environ.get("ANTHROPIC_API_KEY")
                api_key=anthropic_api_key,
            )

            message = client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1000,
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": usercontent
                            }
                        ]
                    }
                ]
            )

            responsetext=message.content[0].text
            #print(responsetext)
            
            for j in range(1):
            #print(aa)
                temp1='blank'
                temp2=responsetext.split('=')[1]
                temp3='blank'
                temp4='blank'
                temp5='blank'
                
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql, (str(eid[i]),temp1,temp2,temp3,temp4,temp5))
                        conn.commit()

        except:
            temp1=np.nan
            temp2=np.nan
            temp3=np.nan
            temp4=np.nan
            temp5=np.nan
            
    else:
        temp1=np.nan
        temp2=np.nan
        temp3=np.nan
        temp4=np.nan
        temp5=np.nan
        


In [ ]:
responsenumnumnum=1
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    input_text=rand_df['input_text']
    ############################
    table_name="20241029_sonnet_eGFR4"
    iterations=1
    upto=20000
    ############################
    for temper in [0.0]:
        time1=datetime.datetime.now()
        while True:
            toggle=0
            for i in range(iterations):   
                newtablaname=table_name + '_' + str(int(experimentnum)) + '_' + str(i)
                try:
                    sql_createtable="CREATE TABLE [" + newtablaname +"""] 
                    (
                        eid    NVARCHAR(20),
                        system    NVARCHAR(max),
                        score      NVARCHAR(max),
                        category     NVARCHAR(max) ,
                        framingham  NVARCHAR(20),
                        ACC_AHA  NVARCHAR(20)
                    )

                    """
                    conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                    with conn:
                        with conn.cursor() as cur:
                            cur.execute(sql_createtable)
                            conn.commit()
                    time.sleep(1)  
                except:
                    pass

                sql_statement="select * from ["+ newtablaname + "]"
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                datasql = pd.read_sql(sql=sql_statement, con=conn)
                datasql=datasql.astype({'eid':int})
                cnt_df=datasql.groupby('eid')['score'].count()
                tempcount=0
                for j in range(upto):
                    try:
                        cnt=cnt_df[j]
                        if cnt<responsenumnumnum:
                            addcnt=int(responsenumnumnum-cnt)
                            sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                            ChatGPT_main(j,temper,addcnt)
                            tempcount+=1

                    except:
                        sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                        ChatGPT_main(j,temper,responsenumnumnum)
                        tempcount+=1
                    if j%100==0:
                        print(j)

                if tempcount==0:
                    toggle=1
            if toggle==1:
                break
            time.sleep(20)

        time.sleep(20)


        time2=datetime.datetime.now()
        print(time2-time1)
        #print('except_else_num = ',except_else_num)

In [ ]:
gpt_cindex=[]
validnum=[]
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    #rand_df=pd.read_csv('20241023_eGFR_synthetic_20000.csv')
    input_text=rand_df['input_text']
    ############################
    #table_name="20241029_GEMINI_eGFR3"
    iterations=1
    upto=20000
    ############################
    resultsdf=pd.DataFrame(columns=['validnum','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking','avg_gpt_score','c-index gpt','c-index acc/aha','c-index framingham'])


    for temper in [0.0]:
        templist=[]
        print('=========temperature========',temper)
        for i in range(iterations):
            newtablename=table_name + '_' + str(int(experimentnum)) + '_' + str(i)

            sql_statement="select * from ["+ newtablename + "]"
            conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)


        for j in range(iterations):
            dat=globals()['data{}'.format(j)]

            eid=dat['eid']
            score_gpt=dat['score']
            category=dat['category']
            score_framingham=dat['framingham']
            score_acc_aha=dat['ACC_AHA']

            score_gpt2=[]

            for i in range(len(score_gpt)):
                try:
                    score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                        except:
                            try:
                                score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                            except:
                                score_gpt2.append(np.nan)
                                #score_gpt2.append(float(score_framingham[i]))
                                #print(score_gpt[i],'====')


        data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
        data_final['eid']=data_final['eid'].astype(int)
        print(len(data_final))
        data_final=data_final.dropna()
        data_final.reset_index(inplace=True,drop=True)


In [ ]:
#data_final = data_final.groupby('eid')['score'].mean().reset_index()
df_merged=pd.merge(data_final,rand_df,how='inner',on='eid')

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
X=df_merged[columnslisttemp]
y=df_merged['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(columnslisttemp)
print(line_fitter.coef_)


In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_sonnet_eGFR_R.csv',index=False)

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
df_merged_temp=df_merged[df_merged['sex']==1]

X=df_merged_temp[columnslisttemp]
X.drop(['sex'],inplace=True,axis=1)
y=df_merged_temp['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(columnslisttemp)
print(line_fitter.coef_)

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_sonnet_eGFR_R_male.csv',index=False)

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
df_merged_temp=df_merged[df_merged['sex']==0]

X=df_merged_temp[columnslisttemp]
X.drop(['sex'],inplace=True,axis=1)
y=df_merged_temp['score']

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(columnslisttemp)
print(line_fitter.coef_)

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_sonnet_eGFR_R_female.csv',index=False)

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
df_merged_temp=df_merged[df_merged['sex']==1]

X=df_merged_temp[columnslisttemp]
X.drop(['sex'],inplace=True,axis=1)
X['log_age'] = np.log(X['age'])
X['log_Cr'] = np.log(X['Cr'])
X.drop(['age','Cr'],inplace=True,axis=1)
y=df_merged_temp['score']
y = np.log(y)

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(X.columns)
print(line_fitter.coef_)

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_sonnet_eGFR_R_male_log.csv',index=False)

In [ ]:
columnslisttemp=list(df_merged.columns)
columnslisttemp=columnslisttemp[2:]
columnslisttemp=columnslisttemp[:-1]


#print(columnslisttemp)
#X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
df_merged_temp=df_merged[df_merged['sex']==0]

X=df_merged_temp[columnslisttemp]
X.drop(['sex'],inplace=True,axis=1)
X['log_age'] = np.log(X['age'])
X['log_Cr'] = np.log(X['Cr'])
X.drop(['age','Cr'],inplace=True,axis=1)
y=df_merged_temp['score']
y = np.log(y)

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(X.columns)
print(line_fitter.coef_)

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_sonnet_eGFR_R_female_log.csv',index=False)

In [ ]:
df_merged_copy=df_merged.copy()

df_merged_m=df_merged_copy[df_merged_copy['sex']==1]
df_merged_f=df_merged_copy[df_merged_copy['sex']==0]
df_merged_m.reset_index(inplace=True,drop=True)
df_merged_f.reset_index(inplace=True,drop=True)

df_merged_m['Cr_min'] = df_merged_m['Cr'].apply(lambda x: min(x / 0.9, 1))
df_merged_m['log_Cr_min_m']=np.log(df_merged_m['Cr_min'])
df_merged_m['log_Cr_min_f']=0
df_merged_m['Cr_max'] = df_merged_m['Cr'].apply(lambda x: max(x / 0.9, 1))
df_merged_m['log_Cr_max']=np.log(df_merged_m['Cr_max'])

df_merged_f['Cr_min'] = df_merged_f['Cr'].apply(lambda x: min(x / 0.7, 1))
df_merged_f['log_Cr_min_m']=0
df_merged_f['log_Cr_min_f']=np.log(df_merged_f['Cr_min'])
df_merged_f['Cr_max'] = df_merged_f['Cr'].apply(lambda x: max(x / 0.7, 1))
df_merged_f['log_Cr_max']=np.log(df_merged_f['Cr_max'])
                        
df_merged_copy=pd.concat([df_merged_m,df_merged_f])     
df_merged_copy.reset_index(inplace=True,drop=True)

df_merged_copy['log_score']=np.log(df_merged_copy['score'])

df_merged_copy['female'] = df_merged_copy['sex'].apply(lambda x: 1 if x == 0 else 0)


y=df_merged_copy['log_score']
Xcolumns=['log_Cr_min_m','log_Cr_min_f','log_Cr_max','age','black','female']


X=df_merged_copy[Xcolumns]

line_fitter = LinearRegression()
line_fitter.fit(X, y)
print(X.columns)
print(line_fitter.coef_)

In [ ]:
X2=X.copy()
X2['y']=y
X2.to_csv('20241029_sonnet_eGFR_R_CKD_EPI.csv',index=False)